<a href="https://colab.research.google.com/github/l642/Audio-Tampering-Detection/blob/main/tryAllModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

data=pd.read_csv("/content/spliced_3.csv")
data.head()
#data["label"].value_counts()


,fname,label,fold
0,jockson1.wav,1,1
1,jockson10.wav,1,1
2,jockson100.wav,1,1
3,jockson1001.wav,1,1
4,jockson1002.wav,1,1


In [ ]:
from librosa import display
import librosa
y,sr=librosa.load("/content/jockson1.wav")
print(y, sr)
mfccs = librosa.feature.mfcc(y, sr, n_mfcc=40)
melspectrogram =librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000)
chroma_stft=librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40)
chroma_cq =librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=40)
chroma_cens =librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=40)
melspectrogram.shape,chroma_stft.shape,chroma_cq.shape,chroma_cens.shape,mfccs.shape


[0. 0. 0. ... 0. 0. 0.] 22050


((40, 463), (40, 463), (40, 463), (40, 463), (40, 463))

In [ ]:
from librosa import display
import librosa
y,sr=librosa.load("/content/jockson1.wav")
print(y.shape, sr)
stft=np.mean(librosa.stft(y))
#stft.shape
#stft=np.mean(librosa.stft(y,sr))
stft=np.mean(librosa.stft(y).T,axis=0)
#mfccs = librosa.feature.mfcc(y, sr, n_mfcc=200)
D = np.abs(librosa.stft(y))
stft.shape
#D.shape
#mfccs.shape
#features=np.reshape(np.vstack((stft)),(474575))
#features.shape


(236919,) 22050


(1025,)

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
librosa.display.specshow(mfccs, x_axis='time')
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()

In [ ]:
#processing with all features
x_train=[]
x_test=[]
y_train=[]
y_test=[]
path="/content/drive/My Drive/spliced3/fold"
for i in tqdm(range(len(data))):
    fold_no=str(data.iloc[i]["fold"])
    file=data.iloc[i]["fname"]
    label=data.iloc[i]["label"]
    filename=path+fold_no+"/"+file
    y,sr=librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    chroma_stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=40).T,axis=0)
    features=np.reshape(np.vstack((mfccs,melspectrogram,chroma_stft,chroma_cq,chroma_cens)),(200))
    if(fold_no!='3'):
      x_train.append(features)
      y_train.append(label)
    else:
      x_test.append(features)
      y_test.append(label)
    


In [ ]:
#processing with any of the single feature
x_train=[]
x_test=[]
y_train=[]
y_test=[]
path="/content/drive/My Drive/spliced3/fold"
for i in tqdm(range(len(data))):
    fold_no=str(data.iloc[i]["fold"])
    file=data.iloc[i]["fname"]
    label=data.iloc[i]["label"]
    filename=path+fold_no+"/"+file
    y,sr=librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    #melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=100,fmax=8000).T,axis=0)
    #stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    stft=np.mean(librosa.stft(y).T,axis=0)
    #chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=100).T,axis=0)
    #chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=100).T,axis=0)
    #features=np.reshape(np.vstack((mfccs,melspectrogram,chroma_stft,chroma_cq,chroma_cens)),(500))
    if(fold_no!='2'):
      x_train.append(stft)
      y_train.append(label)
    else:
      x_test.append(stft)
      y_test.append(label)
    


100%|██████████| 8300/8300 [58:50<00:00,  2.35it/s]


In [ ]:
len(x_train)+len(x_test)

8300

In [ ]:
#Data reshaping for ANN(1D)
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
x_train.shape,x_test.shape,y_train.shape,y_test.shape


((4780, 1025), (3520, 1025), (4780,), (3520,))

In [ ]:
#Data reshaping for CNN(2D)
x_train=np.reshape(x_train,(x_train.shape[0], 463,1025,1))
x_test=np.reshape(x_test,(x_test.shape[0], 200,1,1))

In [ ]:
#Data reshaping for LSTM(3D)
x_train=np.reshape(x_train,(x_train.shape[0], 1,200,1))
x_test=np.reshape(x_test,(x_test.shape[0], 1,200,1))
x_train.shape,x_test.shape
x_train.shape,x_test.shape,y_train.shape,y_test.shape


In [ ]:
#test data size same for all models
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)
y_train.shape,y_test.shape

((4780, 2), (3520, 2))

In [ ]:
#model building
from keras import Sequential
from keras.layers import LSTM ,Conv2D,MaxPooling2D,Flatten
from keras.layers import Dense,Dropout,Activation

In [ ]:
model=Sequential()

In [ ]:
#ANN Model
model.add(Dense(units=256,activation='relu',input_dim=1025))
model.add(Dropout(0.4))
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=2,activation='softmax'))

In [ ]:
#CNN Model
model.add(Conv2D(64,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(200,1,1)))
model.add(MaxPooling2D(padding="same"))

model.add(Conv2D(128,kernel_size=5,strides=1,padding="same",activation="relu"))
model.add(MaxPooling2D(padding="same"))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(2,activation="softmax"))

In [ ]:
#LSTM Model
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(200,1)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(2))


In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),batch_size=500)


Epoch 1/50
10/10 [==============================] - 1s 51ms/step - loss: 0.5697 - accuracy: 0.7900 - val_loss: 1.7358 - val_accuracy: 0.0000e+00
Epoch 2/50
10/10 [==============================] - 0s 35ms/step - loss: 0.4756 - accuracy: 0.8159 - val_loss: 1.5556 - val_accuracy: 0.0000e+00
Epoch 3/50
10/10 [==============================] - 0s 34ms/step - loss: 0.4460 - accuracy: 0.8159 - val_loss: 1.5762 - val_accuracy: 0.0000e+00
Epoch 4/50
10/10 [==============================] - 0s 35ms/step - loss: 0.4324 - accuracy: 0.8159 - val_loss: 1.5708 - val_accuracy: 0.0000e+00
Epoch 5/50
10/10 [==============================] - 0s 35ms/step - loss: 0.4222 - accuracy: 0.8159 - val_loss: 1.5768 - val_accuracy: 0.0000e+00
Epoch 6/50
10/10 [==============================] - 0s 35ms/step - loss: 0.4154 - accuracy: 0.8159 - val_loss: 1.5184 - val_accuracy: 0.0000e+00
Epoch 7/50
10/10 [==============================] - 0s 35ms/step - loss: 0.4038 - accuracy: 0.8159 - val_loss: 1.5547 - val_accura

In [ ]:
train_loss_score=model.evaluate(x_train,y_train)
test_loss_score=model.evaluate(x_test,y_test)
print(train_loss_score)
print(test_loss_score)

110/110 [==============================] - 0s 2ms/step - loss: 1.7482 - accuracy: 0.6253
[0.032061394304037094, 0.9907949566841125]
[1.7482277154922485, 0.6252840757369995]


In [ ]:
#predict first 50 audios in the test set
y_predicted=model.predict(x_test)

In [ ]:
import tensorflow as tf
y_predicted_labels = [np.argmax(i) for i in y_predicted]
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_predicted_labels)
cm

In [ ]:
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')